In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../src'

/media/apfriend/sabrent/ucsd/classes/cse151b/kaggle/src


In [33]:
import os
import sys
import pandas as pd
import random
import warnings
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
from glob import glob
from time import sleep
import matplotlib.pyplot as plt
import random
import multiprocessing
from tqdm import tqdm
import seaborn as sns
from IPython.display import display

#####LOCAL SCRIPTS FROM src#####
import utility
import load_data as load_data
import models.lstm as lstm
import models.linear as linear
import models.physics as physics
from ArgoverseDataset import ArgoverseDataset
import run_model as run_model

In [3]:
train_path='../data/train-data'
val_path='../data/validation-data'
test_path='../data/test-data'
out_path='../data/out-data'
fig_path='../latex/figures/train-loss'
model_path='../trained-models'
device='cuda:0'

In [4]:
test_inp, test_out=load_data.get_test_data(test_path, 64)
test_inp, test_out=test_inp.to(device), test_out.to(device)
test_inp.shape, test_out.shape

(torch.Size([64, 60, 19, 4]), torch.Size([64, 60, 30, 4]))

In [202]:
test_pred=physics.avg_velocity_model(test_inp)
test_pred.shape

torch.Size([64, 60, 30, 2])

In [203]:
pos_idx=torch.tensor([0,1]).to(device)
test_out_pos=torch.index_select(test_out, 3, pos_idx)
utility.rmse(test_pred, test_out_pos)

tensor(28.2396, device='cuda:0')

In [201]:
test_pred.device

device(type='cpu')

In [5]:
velocity_idx=torch.tensor([2,3]).to(device)
velocity_tensor=torch.index_select(test_inp, 3, velocity_idx)
test_idx=torch.tensor([0,1]).to(device)
torch.index_select(velocity_tensor, 3, test_idx)

tensor([[[[-7.5388e+00, -7.0237e+00],
          [-7.8586e+00, -7.3268e+00],
          [-8.6117e+00, -8.0371e+00],
          ...,
          [-8.5952e+00, -7.9826e+00],
          [-7.8026e+00, -7.2253e+00],
          [-7.7894e+00, -7.2213e+00]],

         [[-5.4084e+00, -6.6081e+00],
          [-5.8438e+00, -5.4105e+00],
          [-5.8735e+00, -5.2767e+00],
          ...,
          [-3.4089e+00, -2.8355e+00],
          [-4.5769e+00, -4.2047e+00],
          [-3.9096e+00, -3.5916e+00]],

         [[-9.8587e+00, -9.3302e+00],
          [-1.0427e+01, -8.7073e+00],
          [-9.2263e+00, -7.8562e+00],
          ...,
          [-7.6161e+00, -7.1806e+00],
          [-8.6910e+00, -6.7382e+00],
          [-6.6854e+00, -6.8684e+00]],

         ...,

         [[ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00],
          ...,
          [ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

     

In [6]:
avg_velocity_tensor, position_tensor=physics.avg_velocity_model(test_inp)#.shape
avg_velocity_tensor.shape, position_tensor.shape

(torch.Size([64, 60, 1, 2]), torch.Size([64, 60, 19, 2]))

In [146]:
test_inp.shape

torch.Size([64, 60, 19, 4])

In [23]:
avg_velocity_tensor

tensor([[[[-7.9601e+00, -7.4163e+00]],

         [[-4.8753e+00, -4.6647e+00]],

         [[-8.4215e+00, -7.8060e+00]],

         ...,

         [[ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00]]],


        [[[ 9.6260e+00,  8.0349e+00]],

         [[ 9.8527e+00,  8.3192e+00]],

         [[ 8.9582e+00,  7.5443e+00]],

         ...,

         [[ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00]]],


        [[[-5.6850e+00, -8.0029e-01]],

         [[-2.5745e+00, -6.1194e+00]],

         [[-3.7473e-01,  4.7432e-02]],

         ...,

         [[ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00]]],


        ...,


        [[[ 8.3591e+00,  6.9920e+00]],

         [[ 7.1196e+00,  5.9848e+00]],

         [[ 1.1606e+01,  1.0595e+01]],

         ...,

         [[ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00]],


In [26]:
avg_velocity_tensor.reshape(64*60, 2)

tensor([[-7.9601, -7.4163],
        [-4.8753, -4.6647],
        [-8.4215, -7.8060],
        ...,
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]], device='cuda:0')

In [27]:
64*60

3840

In [148]:
final_x_pos_tensor,final_y_pos_tensor, avg_vel_tensor=physics.avg_velocity_model(test_inp)
final_x_pos_tensor.shape,final_y_pos_tensor.shape, avg_vel_tensor.shape

(torch.Size([64, 60, 1, 1]),
 torch.Size([64, 60, 1, 1]),
 torch.Size([64, 60, 1, 2]))

In [179]:
final_x_pos_tensor, final_y_pos_tensor, x_pos_delta_tensor, y_pos_delta_tensor=physics.avg_velocity_model(test_inp)
print(final_x_pos_tensor.shape, final_y_pos_tensor.shape, x_pos_delta_tensor.shape, y_pos_delta_tensor.shape)
final_x_pos=torch.cat([final_x_pos_tensor]*30, 1)#.shape
(final_x_pos+x_pos_delta_tensor).shape

torch.Size([3840, 1]) torch.Size([3840, 1]) torch.Size([3840, 30]) torch.Size([3840, 30])


torch.Size([3840, 30])

In [173]:
x_pos_delta_tensor[0]

tensor([  -7.9601,  -15.9202,  -23.8803,  -31.8404,  -39.8005,  -47.7606,
         -55.7207,  -63.6808,  -71.6409,  -79.6010,  -87.5611,  -95.5212,
        -103.4813, -111.4414, -119.4015, -127.3616, -135.3217, -143.2818,
        -151.2419, -159.2020, -167.1621, -175.1222, -183.0823, -191.0424,
        -199.0025, -206.9626, -214.9227, -222.8828, -230.8429, -238.8030],
       device='cuda:0')

In [176]:
final_x_pos_tensor

tensor([[2636.8132],
        [2604.2771],
        [2620.1086],
        ...,
        [   0.0000],
        [   0.0000],
        [   0.0000]], device='cuda:0')

In [156]:
final_x_pos_tensor[0]

tensor([[[2636.8132]],

        [[2604.2771]],

        [[2620.1086]],

        [[2652.4404]],

        [[2587.3477]],

        [[2592.5481]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0000]],

        [[   0.0

In [181]:
x, y=physics.avg_velocity_model(test_inp)
x.shape, y.shape

(torch.Size([3840, 30]), torch.Size([3840, 30]))

In [186]:
x[0]

tensor([2628.8530, 2620.8931, 2612.9329, 2604.9729, 2597.0127, 2589.0527,
        2581.0925, 2573.1323, 2565.1724, 2557.2122, 2549.2522, 2541.2920,
        2533.3320, 2525.3718, 2517.4116, 2509.4517, 2501.4915, 2493.5315,
        2485.5713, 2477.6113, 2469.6511, 2461.6909, 2453.7310, 2445.7708,
        2437.8108, 2429.8506, 2421.8906, 2413.9304, 2405.9702, 2398.0103],
       device='cuda:0')

In [187]:
y[0]

tensor([1125.0094, 1117.5931, 1110.1769, 1102.7606, 1095.3444, 1087.9281,
        1080.5118, 1073.0956, 1065.6793, 1058.2631, 1050.8468, 1043.4305,
        1036.0143, 1028.5980, 1021.1818, 1013.7656, 1006.3493,  998.9330,
         991.5168,  984.1005,  976.6843,  969.2680,  961.8517,  954.4355,
         947.0192,  939.6030,  932.1867,  924.7704,  917.3542,  909.9379],
       device='cuda:0')

In [185]:
torch.cat([x,y],dim=1)[0]

tensor([2628.8530, 2620.8931, 2612.9329, 2604.9729, 2597.0127, 2589.0527,
        2581.0925, 2573.1323, 2565.1724, 2557.2122, 2549.2522, 2541.2920,
        2533.3320, 2525.3718, 2517.4116, 2509.4517, 2501.4915, 2493.5315,
        2485.5713, 2477.6113, 2469.6511, 2461.6909, 2453.7310, 2445.7708,
        2437.8108, 2429.8506, 2421.8906, 2413.9304, 2405.9702, 2398.0103,
        1125.0094, 1117.5931, 1110.1769, 1102.7606, 1095.3444, 1087.9281,
        1080.5118, 1073.0956, 1065.6793, 1058.2631, 1050.8468, 1043.4305,
        1036.0143, 1028.5980, 1021.1818, 1013.7656, 1006.3493,  998.9330,
         991.5168,  984.1005,  976.6843,  969.2680,  961.8517,  954.4355,
         947.0192,  939.6030,  932.1867,  924.7704,  917.3542,  909.9379],
       device='cuda:0')

In [191]:
final_pos=torch.zeros(x.shape[0],x.shape[1]*2)
final_pos[:,::2]=x
final_pos[:,1::2]=y
final_pos.shape

torch.Size([3840, 60])

In [194]:
physics.avg_velocity_model(test_inp).shape

torch.Size([64, 60, 30, 2])

In [143]:
last_position_idx=position_tensor.shape[2]-1
last_position_tensor=torch.tensor([last_position_idx]).to(device)
last_pos_tensor=torch.index_select(position_tensor, 2, last_position_tensor)
last_pos_tensor.shape

torch.Size([64, 60, 1, 2])

In [12]:
position_tensor.shape

torch.Size([64, 60, 19, 2])

In [9]:
torch.tensor([2,3]).to(device)

tensor([2, 3], device='cuda:0')

In [128]:
x_idx=torch.tensor([0]).to(device)
y_idx=torch.tensor([1]).to(device)
x_position_tensor=torch.index_select(position_tensor, 3, x_idx)
y_position_tensor=torch.index_select(position_tensor, 3, y_idx)
x_position_tensor.unsqueeze(3).shape

torch.Size([64, 60, 19, 1, 1])

In [135]:
time_tensor=torch.FloatTensor([list(range(1,31))]).to(device)
# avg_velocity_tensor.reshape(64*60, 2)*time_tensor

# torch.dot(avg_velocity_tensor.reshape(64*60, 2)*time_tensor.shape)
time_tensor=time_tensor
avg_velocity_tensor=avg_velocity_tensor.reshape(64*60,2)
# avg_velocity_tensor.shape, time_tensor.shape
avg_x_vel_tensor=torch.index_select(avg_velocity_tensor, 1, x_idx)
avg_y_vel_tensor=torch.index_select(avg_velocity_tensor, 1, y_idx)
avg_x_vel_tensor.shape, avg_y_vel_tensor.shape, time_tensor.shape

(torch.Size([3840, 1]), torch.Size([3840, 1]), torch.Size([1, 30]))

In [142]:
x_pos_delta_tensor=torch.tensordot(avg_x_vel_tensor, time_tensor, dims=([1],[0])).reshape(64,60,1,30)
y_pos_delta_tensor=torch.tensordot(avg_y_vel_tensor, time_tensor, dims=([1],[0])).reshape(64,60,1,30)
x_pos_delta_tensor.shape, y_pos_delta_tensor.shape

(torch.Size([64, 60, 1, 30]), torch.Size([64, 60, 1, 30]))

In [144]:
pos_delta_tensor=torch.tensordot(avg_velocity_tensor, time_tensor, dims=([1],[0]))#[0:2]#.shape
pos_delta_tensor.shape, last_pos_tensor.shape

(torch.Size([3840, 30]), torch.Size([64, 60, 1, 2]))

In [123]:
x_final_pos=torch.concat([last_post_tensor]*30)

tensor([[ -15.3764,  -30.7527,  -46.1291,  ..., -430.5380, -445.9144,
         -461.2907],
        [  -9.5400,  -19.0800,  -28.6201,  ..., -267.1205, -276.6606,
         -286.2006],
        [ -16.2275,  -32.4550,  -48.6825,  ..., -454.3698, -470.5973,
         -486.8248],
        ...,
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000]], device='cuda:0')